<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama2/blob/main/ExemplosGeracaoTexto_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Geração de textos usando Llama v2.0 7B 8bit usando Transformers by HuggingFace

Exemplo de uso do modelo de linguagem grande Llama v2.0.
- Analise da geração de textos
- Prompts com textos emparelhados
- Injentando padrões no prompt
- Padrão Persona

**Toda a execução ocorre no Google Colaboratory.**

Pré-requisitos:
- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Notebook de referência:**

https://github.com/guardiaum/tutorial-sbbd2023/blob/main/Prompt_Engineering.ipynb


**Lista dos modelos:**

https://huggingface.co/models


**Artigos referências:**

https://dev.to/nithinibhandari1999/how-to-run-llama-2-on-your-local-computer-42g1


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Data e hora de execução

In [ ]:
# Biblioteca de date
from datetime import datetime

data_e_hora_atuais = datetime.now()
data_e_hora_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y %H:%M:%S')

print(data_e_hora_em_texto)

21/04/2024 09:36:22


## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Biblioteca do sistema
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Versão Python

In [ ]:
# Biblioteca do sistema
import sys

print("Versão Python:", sys.version)

Versão Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

# 1 - Instalação das bibliotecas

O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

https://huggingface.co/blog/4bit-transformers-bitsandbytes

https://pypi.org/project/bitsandbytes/

In [ ]:
!pip install bitsandbytes==0.43.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

https://pypi.org/project/accelerate/

In [ ]:
!pip install accelerate==0.29.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00


A Biblioteca A Biblioteca Transformers fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração para Processamento de linguagem natural, Visão computacional, Áudio, etc.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
!pip install -U transformers==4.40.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 50.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


A Biblioteca huggingface-cli fornece vários comandos para interagir com o Hugging Face Hub a partir da linha de comando. Um desses comandos é o login, que permite aos usuários se autenticarem no Hub usando suas credenciais.

In [ ]:
#!pip install huggingface-hub==0.20.3

## Versão bibliotecas instaladas

In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==0.29.3
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.2.0
bitsandbytes==0.43.1
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.1
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.

# 2 - Carregando o LLM



## 2.1 - Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o token de acesso toda vez que for executar o notebook preencha o método save_token.

Crie a variável 'HF_TOKEN' com o valor do **Access Token do HuggingFace**. Abra o Google Colab e navegue até a nova seção 'Secrets' na barra lateral e adicione a variável.

In [ ]:
from huggingface_hub.hf_api import HfFolder

if IN_COLAB:

    from google.colab import userdata

    # ACESS_TOKEN = "<valor_do_acess_token"
    ACCESS_TOKEN  = userdata.get('HF_TOKEN')

    HfFolder.save_token(ACCESS_TOKEN)

Mostrando o usuário conectado

In [ ]:
!huggingface-cli whoami

osmarbraz


## 2.2 - Nome do modelo de linguagem

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf

In [ ]:
#nome_modelo = "meta-llama/Llama-2-7b-hf"
nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

# Não carrega por falta de memória no google colab
#nome_modelo = "meta-llama/Llama-2-13b-hf"
#nome_modelo = "meta-llama/Llama-2-13b-chat-hf"

# Não carrega por falta de memória e espaço em disco no google colab
#nome_modelo = "meta-llama/Llama-2-70b-hf"
#nome_modelo = "meta-llama/Llama-2-70b-chat-hf"

## 2.3 - Carrega o tokenizador

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador meta-llama/Llama-2-7b-chat-hf da comunidade...


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Tamanho do vocabulário

In [ ]:
print(len(tokenizer))

32000


## 2.4 - Carregando o LLM

Carregando o **LLM** da comunidade HuggingFace.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

Carregamento LLama 2 com 4 bits

In [ ]:
# # Importando as bibliotecas do Modelo
# from transformers import BitsAndBytesConfig, AutoModelForCausalLM
# import torch
# import time

# # Guarda o tempo de início do carregamento do modelo
# tempo_inicio = time.time()

# # Carregando o Modelo da comunidade
# print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# # BitsAndBytes é um framework com funções customizadas para
# # otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
# quantization_config = BitsAndBytesConfig(
#    load_in_4bit=True, # Habilita a quantização de 4 bits para comprimir o modelo
#    bnb_4bit_quant_type="nf4", # Define o tipo de dados de quantização nas camadas (`fp4` e `nf4`).
#    bnb_4bit_use_double_quant=True, # Quantização aninhada, onde as constantes de quantização da primeira quantização são quantizadas novamente.
#    bnb_4bit_compute_dtype=torch.bfloat16 # # Os gradientes dos pesos são computados em 16-bit. Define o tipo computacional que pode ser diferente do tempo de entrada. Por exemplo, as entradas podem ser fp32, mas a computação pode ser definida como bf16 para acelerações.
# )

# # Carrega o modelo
# model = AutoModelForCausalLM.from_pretrained(nome_modelo,
#                                              #torch_dtype=torch.float16, #default
#                                              trust_remote_code=True, # Carrega de um repositório confiável
#                                              quantization_config=quantization_config,
#                                              device_map="auto"
#                                              )

# # Coloca o modelo e modo avaliação
# model.eval()

# # Aumentar a velocidade
# # https://huggingface.co/docs/transformers/main/perf_torch_compile
# model = torch.compile(model)

# print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregamento LLama 2 com 8 bits

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# BitsAndBytes é um framework com funções customizadas para
# otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
quantization_config = BitsAndBytesConfig(
   load_in_8bit=True, # Habilita a quantização de 8 bits
)

# Carrega o modelo
model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True, # Carrega de um repositório confiável
                                             quantization_config=quantization_config,
                                             device_map="auto"
                                             )

# Coloca o modelo e modo avaliação
model.eval()

# Aumentar a velocidade
# https://huggingface.co/docs/transformers/main/perf_torch_compile
model = torch.compile(model)

print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo meta-llama/Llama-2-7b-chat-hf da comunidade...


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Tempo de carregamento do modelo LLM:  0:02:52 (h:mm:ss)


In [ ]:
print(model)

OptimizedModule(
  (_orig_mod): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
            (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
            (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm(

In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": false,
    "_load_in_8bit": true,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_

In [ ]:
print(model.config.max_position_embeddings)

4096


Tamanho do vocabulário

In [ ]:
print(model.config.vocab_size)

32000


## 2.5 - Configuração da geração de texto

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig

# Instância as configurações do modelo
generation_config = GenerationConfig.from_pretrained(nome_modelo)

print("GenerationConfig antes:\n",generation_config)
generation_config.max_new_tokens = 512 #Preenche até um comprimento máximo especificado com o argumento max_length ou até o comprimento de entrada máximo aceitável para o modelo se esse argumento não for fornecido.
#generation_config.max_length = 4096 # (Default 4096)
# Se do_sample é true setar temperature e top_p, caso contrário se do_sample é false remover temperature e top_p.
generation_config.do_sample = True # (Default True) Se definido como True, este parâmetro permite estratégias de decodificação como amostragem multinomial, amostragem multinomial de busca de feixe, amostragem Top-K e amostragem Top-p. Todas essas estratégias selecionam o próximo token da distribuição de probabilidade em todo o vocabulário com vários ajustes específicos da estratégia.
generation_config.temperature = 0.1 # (Default 0.6) A temperatura é um parâmetro que controla a aleatoriedade da saída do LLM. Uma temperatura mais alta resultará em um texto mais criativo e imaginativo, enquanto uma temperatura mais baixa resultará em um texto mais preciso e factual.
#generation_config.top_k = 3  # Top-k diz ao modelo para escolher o próximo token entre os 'k' tokens principais de sua lista, classificados por probabilidade.
#generation_config.top_p = 0.9 # (Default 0.9) Top-p é mais dinâmico que top-k e é frequentemente usado para excluir resultados com probabilidades mais baixas. Portanto, se você definir p como 0,75, excluirá os 25% inferiores dos resultados prováveis.
#generation_config.repetition_penalty = 1.20 # Penaliza a repetição e visa evitar frases que se repetem sem nada de realmente interessante.
#generation_config.num_return_sequences=1, # Retorna uma única sentença da saída.
generation_config.pad_token_id=generation_config.eos_token_id
print("GenerationConfig depois:\n",generation_config)

GenerationConfig antes:
 GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}

GenerationConfig depois:
 GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 512,
  "pad_token_id": 2,
  "temperature": 0.1,
  "top_p": 0.9
}



# 3 - Analisando a geração de textos



## 3.1 - Geração de texto


Define o documento

In [ ]:
# Define o documento base
documento = "Como empilhar elementos em uma pilha?"
#documento = "How to push elements in a stack"
#documento = "O comando SQL para extrair todos os usuários cujo nome começa com A é:"
#documento = "Bom dia professor, tudo bem ?"
# documento = "The SQL command to extract all the users whose name starts with A is:"
#documento = "How to push elements in a stack"
#documento = "Write code for finding the prime number in python ?"
# documento = "Escrever código para encontrar o número primo em python?"

# Prepara o prompt para enviar ao modelo realizando sua tokenização
# Se pt for especificado, ele retornará tensores em vez de lista de inteiros python e tokenizará os documentos
input = tokenizer(documento, return_tensors="pt")

# Mostra os tokens com seus índices
i = 0
for tup in input.input_ids[0]:
    # print(tup.item())
    print("{} {}".format(i, tokenizer.convert_ids_to_tokens(tup.item())))
    i= i + 1

0 <s>
1 ▁Como
2 ▁emp
3 il
4 har
5 ▁elementos
6 ▁em
7 ▁uma
8 ▁pil
9 ha
10 ?


Configura e envia o texto ao LLM

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
input_ids = input["input_ids"].to(model.device)

# Envia a prompt preparado ao modelo
outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config, # Passa as configurações da geração de texto para o modelo
        # https://huggingface.co/docs/transformers/v4.34.1/en/main_classes/output#transformers.utils.ModelOutput
        return_dict_in_generate=True, # A geração retorna um dicionário com 'last_hidden_state' o estado ocultos da última camada do modelo, 'hidden_states' estados ocultos do modelo na saída de cada camada mais as saídas dos embeddings iniciais opcionais  e 'attention' com os psos de atenção após o softmax de atenção, usado para calcular a média ponderada nas cabeças de autoatenção.
        output_scores=True, # Retorna as pontuações de previsão do modelo.
        max_new_tokens=256 # O número máximo de tokens a serem gerados, ignorando o número de tokens no prompt.
    )

In [ ]:
print(len(outputs))

3


In [ ]:
print(outputs)

GenerateDecoderOnlyOutput(sequences=tensor([[    1, 17295,  3710,   309,  8222, 29290,   953,  3672,  8230,  2350,
         29973,    13,    13, 29963,   542, 30037, 13279,  3710,   309,  8222,
         29290,   953,  3672,  8230,  2350,   502,  1743,   263,  2090,  2340,
           421,  5910,  2555,  2123,   421,  1202, 13595,   319,  2090,  2340,
           421,  5910,  2555,   594, 15353, 29874,  1922,  1543, 29877,   818,
          3810, 11558,  1146,  8230,  2350, 29892, 22085,  5361,   263,  2090,
          2340,   421,  1202,  2555,   594, 15353, 29874,  1922,  1543, 29877,
           818,  3810, 20773,  1146,  8230,  2350, 29889,    13,    13,  1252,
         13141,   316,  1986, 28336,   421,  5910,  2555,  1702,  3710,   309,
          8222, 29290,   953,  3672,  8230,  2350, 29901,    13, 28956,    13,
          1026,  5096,   353, 13769,    13,  1429, 29889,  5910, 29898, 29896,
           416,    13,  1429, 29889,  5910, 29898, 29906,   416,    13,  1429,
         29889, 

Decodificação

Nossa etapa de geração gera uma matriz de tokens em vez de palavras. Para converter esses tokens em palavras, precisamos realizar sua decodificação.

In [ ]:
# Mostra o resultado
for s in outputs.sequences:
  # Decodifica a saída
  # skip_special_tokens=True retira os tokens especiais da saída da decodificação
  output = tokenizer.decode(s, skip_special_tokens=True)
  print(output)

Como empilhar elementos em uma pilha?

Você pode empilhar elementos em uma pilha usando a função `push()` ou `add()`. A função `push()` adiciona um elemento à parte superior da pilha, enquanto a função `add()` adiciona um elemento à parte inferior da pilha.

Exemplo de como usar `push()` para empilhar elementos em uma pilha:
```
let stack = [];
stack.push(1);
stack.push(2);
stack.push(3);
console.log(stack); // [1, 2, 3]
```
Exemplo de como usar `add()` para empilhar elementos em uma pilha:
```
let stack = [];
stack.add(1);
stack.add(2);
stack.add(3);
console.log(stack); // [1, 2, 3]
```
Você também pode usar a função `peek()` para ver o elemento mais recentemente adicionado à pilha sem remove-lo.
```
let stack = [];
stack.push(1);
stack.push(2);
stack.


In [ ]:
# Mostra o resultado
# skip_special_tokens=True retira os tokens especiais da saída da decodificação
print(tokenizer.decode(outputs.sequences[0], skip_special_tokens=True))

Como empilhar elementos em uma pilha?

Você pode empilhar elementos em uma pilha usando a função `push()` ou `add()`. A função `push()` adiciona um elemento à parte superior da pilha, enquanto a função `add()` adiciona um elemento à parte inferior da pilha.

Exemplo de como usar `push()` para empilhar elementos em uma pilha:
```
let stack = [];
stack.push(1);
stack.push(2);
stack.push(3);
console.log(stack); // [1, 2, 3]
```
Exemplo de como usar `add()` para empilhar elementos em uma pilha:
```
let stack = [];
stack.add(1);
stack.add(2);
stack.add(3);
console.log(stack); // [1, 2, 3]
```
Você também pode usar a função `peek()` para ver o elemento mais recentemente adicionado à pilha sem remove-lo.
```
let stack = [];
stack.push(1);
stack.push(2);
stack.


## 3.2 - Geração de texto com prompt

https://medium.com/@princekrampah/langchain-building-language-model-applications-c54cfe7219cb

https://github.com/awesome-chatgpt-prompts/awesome-chatgpt-prompts-github

Define o documento

In [ ]:
# Define o documento base
documento = "Como empilhar elementos em uma pilha?"

Cria o prompt substituindo o parâmetro {documento} pelo conteúdo da variável documento.

In [ ]:
prompt = f"""Pergunta: {documento}
Resposta: Responda passo a passo.
"""

Configura e envia o prompt ao LLM

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
input_ids = input["input_ids"].to(model.device)

# Prepara o prompt para enviar ao modelo realizando sua tokenização
inputs = tokenizer(prompt, return_tensors="pt")
# Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
input_ids = inputs["input_ids"].to(model.device)

# Envia a prompt preparado ao modelo
output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    #max_new_tokens=512
)

Decodificação

Nossa etapa de geração gera uma matriz de tokens em vez de palavras. Para converter esses tokens em palavras, precisamos realizar sua decodificação.

In [ ]:
# Mostra o resultado
for s in outputs.sequences:
  # Decodifica a saída
  # skip_special_tokens=True retira os tokens especiais da saída da decodificação
  output = tokenizer.decode(s, skip_special_tokens=True)
  print(output)

Como empilhar elementos em uma pilha?

Você pode empilhar elementos em uma pilha usando a função `push()` ou `add()`. A função `push()` adiciona um elemento à parte superior da pilha, enquanto a função `add()` adiciona um elemento à parte inferior da pilha.

Exemplo de como usar `push()` para empilhar elementos em uma pilha:
```
let stack = [];
stack.push(1);
stack.push(2);
stack.push(3);
console.log(stack); // [1, 2, 3]
```
Exemplo de como usar `add()` para empilhar elementos em uma pilha:
```
let stack = [];
stack.add(1);
stack.add(2);
stack.add(3);
console.log(stack); // [1, 2, 3]
```
Você também pode usar a função `peek()` para ver o elemento mais recentemente adicionado à pilha sem remove-lo.
```
let stack = [];
stack.push(1);
stack.push(2);
stack.


# 4 - Exemplos de tipos de prompts

* **zero-shot (0-shot) prompts - Solicitação direta**

    Usado quando você deseja que o modelo gere uma resposta sem exemplos. Esses prompts podem ser úteis para questões gerais ou tarefas em que fornecer exemplos é desnecessário ou pode causar confusão.

    Use prompts de disparo 0 quando confiar no conhecimento geral do modelo para fornecer uma resposta suficiente.


* **one-shot (1-shot) prompts - Solicitação com um exemplo**

    Forneça um único exemplo do resultado desejado, ajudando a orientar a resposta do modelo. Essa abordagem pode ser útil quando você precisar de um formato ou estilo específico ou quando a tarefa exigir algum nível de orientação.

    Use prompts únicos quando quiser empurrar o modelo na direção certa sem sobrecarregá-lo com vários exemplos.

* **few-shot (N-shot) prompts - Solicitação com vários  exemplos**

    Ofereça vários exemplos, permitindo que o modelo aprenda com várias instâncias. Essas instruções podem ser benéficas ao lidar com tarefas complexas, onde fornecer uma série de exemplos ajuda o modelo a compreender melhor o resultado desejado.

    Use prompts multi-shot quando um único exemplo pode não ser suficiente para orientar o modelo ou quando você deseja demonstrar um padrão ou tendência.


Referências:
https://anilktalla.medium.com/prompt-engineering-1-shot-prompting-283a0b2b1467

https://www.ssw.com.au/rules/shot-prompts/

https://github.com/awesome-chatgpt-prompts/awesome-chatgpt-prompts-github

Repositório de pompts: https://github.com/awesome-chatgpt-prompts/awesome-chatgpt-prompts-github



## 4.1 - Zero-shot - Solicitação direta

In [ ]:
def gerar_prompt_zeroshot(texto):

  return f"""{texto}
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

def avaliarPromptZeroShot(texto):
  # Recupera o prompt
    prompt = gerar_prompt_zeroshot(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        #max_new_tokens=512
    )

    resultado = ""
    # Recupera o resultado
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        resposta = output.split("### Resposta:")[1].strip()
        resultado = resultado + resposta

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

    return resultado

In [ ]:
texto = "Me fale sobre algoritmos."

resultado = avaliarPromptZeroShot(texto)

print(resultado)

Sure, I'd be happy to explain algorithms!

An algorithm is a set of instructions used to solve a particular problem or perform a specific task. It is a well-defined procedure that takes some input and produces a corresponding output. Algorithms can be expressed in various forms, such as natural language, flowcharts, pseudocode, or programming languages.

There are many types of algorithms, including:

1. Sorting algorithms: These algorithms are used to sort data in ascending or descending order, such as sorting a list of numbers or names. Examples of sorting algorithms include Bubble Sort, Selection Sort, and Quick Sort.
2. Searching algorithms: These algorithms are used to find specific data within a collection of data. Examples of searching algorithms include Linear Search, Binary Search, and Hashing.
3. Graph algorithms: These algorithms are used to perform operations on graphs, such as finding the shortest path between two nodes, or determining whether a graph is connected. Example

## 4.2 - One-shot - Solicitação com um exemplo

In [ ]:
def gerar_prompt_oneshot(texto):

  return f"""Conte a quantidade de tokens da sentença. Aqui está um exemplo:
\'Elementos são adicionados e removidos apenas no topo da pilha.\' -> '\11\'
Agora conte a quantidade de tokens da sentença: {texto}'
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

def avaliarPromptOneShot(texto):
  # Recupera o prompt
    prompt = gerar_prompt_oneshot(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        #max_new_tokens=512
    )

    resultado = ""
    # Recupera o resultado
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        resposta = output.split("### Resposta:")[1].strip()
        resultado = resultado + resposta

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

    return resultado

In [ ]:
texto = 'Operação de adição em uma pilha é chamada de push.'

resultado = avaliarPromptOneShot(texto)

print(resultado)

'Elementos são adicionados e removidos apenas no topo da pilha.' -> '	' = 3 tokens
'Operação de adição em uma pilha é chamada de push.' -> 'Operação' = 1 token

Para contar a quantidade de tokens em uma sentença, você pode usar um contador de tokens e incrementá-lo em cada palavra ou grupo de palavras. Em seguida, você pode usar o resultado para calcular a quantidade de tokens na sentença.

No exemplo acima, o contador de tokens começa em 0 e é incrementado em 1 para cada palavra na sentença. Assim, o contador de tokens para a sentença 'Elementos são adicionados e removidos apenas no topo da pilha.' é 3, e o contador de tokens para a sentença 'Operação de adição em uma pilha é chamada de push.' é 1.


## 4.3 - Few-shot - Solicitação com vários exemplos

In [ ]:
def gerar_prompt_fewshot(texto):

  return f"""Conte a quantidade de tokens da sentença. Aqui está um exemplo:
\'Pilha e fila são estruturas de dados.\' -> \'7\'\n
\'Elementos são adicionados e removidos apenas do topo da pilha.\' -> '\10\'
\'Pilhas são fundamentais na computação.\' -> '\5\'
Agora conte a quantidade de tokens da sentença: {texto}
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

def avaliarPromptFewShot(texto):
  # Recupera o prompt
    prompt = gerar_prompt_fewshot(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        #max_new_tokens=512
    )

    resultado = ""
    # Recupera o resultado
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        resposta = output.split("### Resposta:")[1].strip()
        resultado = resultado + resposta

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

    return resultado

In [ ]:
texto = 'Operação de adição em uma pilha é chamada de push.'

resultado = avaliarPromptFewShot(texto)

print(resultado)

8

A quantidade de tokens da sentença é: 8

Então, a quantidade de tokens da sentença 'Operação de adição em uma pilha é chamada de push' é 8.


## 4.4 - Tarefas emparelhadas

In [ ]:
def gerar_prompt_tarefa(texto, entrada=None):
    if entrada:
        return f"""Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{texto}

### Entrada:
{entrada}

### Resposta:"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{texto}

### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

def avaliarTextoTarefa(texto, entrada=None):
    # Recupera o prompt
    prompt = gerar_prompt_tarefa(texto, entrada)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        #max_new_tokens=512
    )

    resultado = ""
    #Recupera o resultado
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        resultado = resultado + saida

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

    return resultado

In [ ]:
texto = 'Fale me algo sobre algoritmos.'

resultado = avaliarTextoTarefa(texto)

print(resultado)

Algoritmos são sequências de passos que um computador segue para resolver um problema específico. Eles são usados em diversas áreas, como processamento de linguagem natural, visão computacional, inteligência artificial e muitas outras. Alguns dos algoritmos mais comuns incluem o algoritmo de busca binária, o algoritmo de Floyd-Warshall e o algoritmo de Dijkstra para encontrar o caminho mais curto em uma rede. Além disso, existem algoritmos para resolver problemas de otimização, como o algoritmo de Gradiente Descent, que é usado para ajustar parâmetros de um modelo de aprendizado automático para minimizar um objetivo funcional.


In [ ]:
texto = 'Dada a fórmula química, calcule a massa molar.'

entrada = 'CaCl2'

resultado = avaliarTextoTarefa(texto, entrada)

print(resultado)

A massa molar de CaCl2 é de aproximadamente 105,6 g/mol.

Raisei a resposta!


In [ ]:
texto = 'Faça quatro perguntas sobre a seguinte passagem:'

entrada = 'A anatomia de uma abelha é bastante intrincada. Tem três partes do corpo: a cabeça, o tórax e o abdômen. A cabeça consiste em órgãos sensoriais, três olhos simples e dois olhos compostos e vários apêndices. O tórax tem três pares de pernas e dois pares de asas, enquanto o abdômen contém a maioria dos órgãos da abelha, incluindo o sistema reprodutivo e o sistema digestivo.'

resultado = avaliarTextoTarefa(texto, entrada)

print(resultado)

Espero que essas perguntas ajudem você a entender melhor a anatomia da abelha:

1. Qual é o nome completo da parte do corpo da abelha que contém os órgãos sensoriais?
2. Qual é o nome completo da parte do corpo da abelha que contém o sistema reprodutivo?
3. Qual é o nome completo da parte do corpo da abelha que contém os apêndices?
4. Qual é o nome completo da parte do corpo da abelha que contém os olhos compostos?


In [ ]:
texto = 'Analise o documento jurídico fornecido e explique os pontos-chave.'

entrada = 'O seguinte é um trecho de um contrato entre duas partes, rotulado como "Empresa A" e "Empresa B": "A Empresa A concorda em fornecer assistência razoável à Empresa B para garantir a precisão das demonstrações financeiras que fornece. Isso inclui permitir à Empresa um acesso razoável ao pessoal e outros documentos que possam ser necessários para a revisão da Empresa B. A Empresa B concorda em manter o documento fornecido pela Empresa A em confiança e não divulgará as informações a terceiros sem a permissão explícita da Empresa A".'

resultado = avaliarTextoTarefa(texto, entrada)

print(resultado)

O documento jurídico fornecido é um contrato entre duas partes, em que a Empresa A concorda em fornecer assistência para garantir a precisão das demonstrações financeiras da Empresa B. A Empresa B, por sua vez, concorda em manter o documento fornecido pela Empresa A em confiança e não divulgará as informações a terceiros sem a permissão explícita da Empresa A. Os pontos-chave deste contrato são:

* A Empresa A concorda em fornecer assistência para garantir a precisão das demonstrações financeiras da Empresa B.
* A Empresa B concorda em manter o documento fornecido pela Empresa A em confiança e não divulgará as informações a terceiros sem a permissão explícita da Empresa A.
* A Empresa A e a Empresa B estão de acordo em respeitar a confidencialidade do documento fornecido.

Espero que essa resposta seja útil! Se você tiver alguma dúvida adicional, por favor não hesite em perguntar.


# 5 - Exemplos de injeção de padrões em prompts

 A injeção de padrões faz ignora filtros ou manipula o LLM usando prompts cuidadosamente elaborados que fazem o modelo ignorar instruções anteriores ou executar ações não intencionais.

 https://medium.com/@austin-stubbs/llm-security-types-of-prompt-injection-d7ad8d7d75a3

Repositório de pompts: https://github.com/awesome-chatgpt-prompts/awesome-chatgpt-prompts-github



## 5.1 - Extração de Informação

In [ ]:
def gerar_promptEI(texto):
    ### texto:
    return f"""TEXTO: {texto}
Dado o texto acima, extraia informações importantes no formato abaixo:
<CHAVE>:<VALOR>
### Resposta: """

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

def avaliarEI(texto):
    # Recupera o prompt
    prompt = gerar_promptEI(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        #max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

avaliarEI(texto)

Resposta: 
* Nome: Alan Mathison Turing
* Data de nascimento: 23 de junho de 1912
* Localização de nascimento: Londres
* Data de falecimento: 7 de junho de 1954
* Localização de falecimento: Wilmslow, Cheshire
* Realizações: matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico
* Contribuições: formalização dos conceitos de algoritmo e computação com a máquina de Turing, considerada um modelo de um computador de uso geral
* Reconhecimento: não foi totalmente reconhecido em seu país de origem durante sua vida por ser homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais.


## 5.2 - Entidade nomeada

In [ ]:
def gerar_promptEN(texto):
    ### texto:
    return f"""Detecte as entidades nomeadas no texto a seguir delimitado por aspas triplas.
Retorne apenas a resposta no formato json com spans(Um array que representa o intervalo de caracteres (índices) nos quais a entidade nomeada ocorre no texto original. O primeiro valor no array é o índice inicial e o segundo é o índice final) das entidades nomeadas com os campos \'entidadeNomeada\', \'tipo\', \'span\'.
Retorne todas as entidades
'''{texto}'''
arquivo no formato json:
### Resposta: """

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    # Se do_sample é true setar temperature e top_p, caso contrário se do_sample é false remover temperature e top_p.
    do_sample=True,
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarEN(texto):
    # Recupera o prompt
    prompt = gerar_promptEN(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        #max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

avaliarEN(texto)

Resposta: 
{
"entidadesNomeadas": [
{
"entidadeNomeada": "Alan Mathison Turing",
"tipo": "Pessoa",
"span": [10, 17]
},
{
"entidadeNomeada": "Londres",
"tipo": "Lugar",
"span": [13, 15]
},
{
"entidadeNomeada": "Wilmslow",
"tipo": "Lugar",
"span": [20, 23]
},
{
"entidadeNomeada": "Cheshire",
"tipo": "Lugar",
"span": [24, 26]
}
]
}

Explanation:

* "Alan Mathison Turing" é uma entidade nomeada com o tipo "Pessoa" e o intervalo de caracteres [10, 17].
* "Londres" é uma entidade nomeada com o tipo "Lugar" e o intervalo de caracteres [13, 15].
* "Wilmslow" é uma entidade nomeada com o tipo "Lugar" e o intervalo de caracteres [20, 23].
* "Cheshire" é uma entidade nomeada com o tipo "Lugar" e o intervalo de caracteres [24, 26].

Note: The `span` field represents the range of characters in the original text where the entity name occurs. The first value in the array is the starting index and the second value is the ending index.


## 5.3 - Análise de sentimentos

### 5.3.1 - Análise de sentimentos 1

In [ ]:
def gerar_promptAS1(texto):

  return f"""Classifique os exemplos a seguir de acordo com as seguintes polaridades Positivo, Negativo e Neutro.
EXEMPLO:\n {texto}
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

def avaliarAS1(texto):
    # Recupera o prompt
    prompt = gerar_promptAS1(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        #max_new_tokens=256
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "1 - Minha Experiência na loja foi incrível."\
        "2 - Eu acho que podiam melhorar o produto."\
        "3 - O atendimento foi horrível!"\
        "4 - Não volto mais."\
        "5 - Recomendo demais a banoffe. É uma delícia!"

avaliarAS1(texto)

Resposta: 
1 - Positivo
2 - Negativo
3 - Negativo
4 - Negativo
5 - Positivo


### 5.3.2 - Análise de sentimentos 2

In [ ]:
def gerar_promptAS2(texto):

  return  f"""EXEMPLO: {texto}
Classifique os exemplos de declarações acima de acordo com as polaridades Positivo, Negativo e Neutro.
Utilize o seguinte formato:\n###DECLARAÇÃO:<DECLARAÇÃO>\n###POLARIDADE:<POLARIDADE>.
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

def avaliarAS2(texto):
    # Recupera o prompt
    prompt = gerar_promptAS2(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        # max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "1 - Minha Experiência na loja foi incrível."\
        "2 - Eu acho que podiam melhorar o produto."\
        "3 - O atendimento foi horrível!"\
        "4 - Não volto mais."\
        "5 - Recomendo demais a banoffe. É uma delícia!"

avaliarAS2(texto)

Resposta: 
1. DECLARAÇÃO: Minha Experiência na loja foi incrível.
POLARIDADE: Positivo.
2. DECLARAÇÃO: Eu acho que podiam melhorar o produto.
POLARIDADE: Negativo.
3. DECLARAÇÃO: O atendimento foi horrível!
POLARIDADE: Negativo.
4. DECLARAÇÃO: Não volto mais.
POLARIDADE: Negativo.
5. DECLARAÇÃO: Recomendo demais a banoffe. É uma delícia!
POLARIDADE: Positivo.


## 5.4 - Pergunta e resposta

In [ ]:
def gerar_promptPR(texto):
    '''
      Alterações no texto e tabulação impedem a geração da resposta.
    '''
    return f"""Dado o texto a seguir: {texto}\n
            Gere quatro questões em língua portuguesa e suas respectivas respostas utilizando o template abaixo.\n
            Preserve a exata formatação do template apresentado: \n
            PERGUNTA:<PERGUNTA>
            RESPOSTA:<RESPOSTA>
            ### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

def avaliarPR(texto):
    # Recupera o prompt
    prompt = gerar_promptPR(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        # max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

# Tempo de processamento: ~2m20s
avaliarPR(texto)

Resposta: 
<RESPOSTA>

            PERGUNTA: Como Alan Turing é considerado o pai da ciência da computação teórica?
            RESPOSTA: Turing proporcionou uma formalização dos conceitos de algoritmo e computação com a máquina de Turing, que pode ser considerada um modelo de um computador de uso geral.

            PERGUNTA: Por que Alan Turing não foi totalmente reconhecido em seu país de origem durante sua vida?
            RESPOSTA: Ele não foi reconhecido em seu país de origem durante sua vida por ser homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais.

            PERGUNTA: Qual é a Lei de Segredos Oficiais mencionada na resposta anterior?
            RESPOSTA: A Lei de Segredos Oficiais é uma lei britânica que proibia a divulgação de informações sobre o trabalho de Turing e outros cientistas da computação durante a Segunda Guerra Mundial.

            PERGUNTA: Qual é o significado da expressão "computação com a máquina de Turing"?
     

# 6 - Exemplos de padrão de pessoa (padrão persona) em prompts

## 6.1 Um matemático

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        # max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva como se fosse um professor de matemática. Me explique no idioma português a importância do teorema de pitágoras.'

avaliarTexto(texto)

Resposta: 
Bem-vindos à minha aula de matemática! Hoje, vamos abordar um dos teoremas mais importantes da história da matemática: o Teorema de Pitágoras.

O Teorema de Pitágoras é um dos principais resultados da geometria euclidiana, que estabelece uma relação matemática entre os lados de um triângulo retângulo. Em resumo, o teorema afirma que, se um triângulo retângulo tem um cateto maior e um cateto menor, então o quadrado da medida do cateto maior é igual à soma dos quadrados dos catetos menores.

Mas, você pode perguntar: "O que é tão importante nesse teorema? Por que é importante estudar esse resultado?" Boa pergunta! A resposta é que o Teorema de Pitágoras tem muitas aplicações práticas em diversas áreas da vida, desde a engenharia até a física.

Para começar, o Teorema de Pitágoras é fundamental na construção de edifícios e outras estruturas. Imagine que você está construindo uma torre, por exemplo. Para saber a altura da torre, você precisa calcular o quadrado da medida do cate

## 6.2 Um advogado

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        # max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva como se fosse um advogado brasileiro especialista em direito penal. '\
        'Pontue de forma resumida as possíveis penas para um caso de lesão corporal leve sem contexto de violência doméstica.'

avaliarTexto(texto)

Resposta: 
Com base nos artigos 20 e 21 do Código Penal Brasileiro, as penas possíveis para um caso de lesão corporal leve sem contexto de violência doméstica são:

* Detenção de até 2 anos e 4 meses;
* Multa de até R$ 1.000,00;
* Fazenda Pública de até R$ 1.000,00;
* Suspensão do direito de exercer determinadas atividades profissionais ou ocupar determinados cargos públicos por um período de até 2 anos e 4 meses.

É importante lembrar que a pena é sempre subjeto às disposições do Código de Processo Penal e do Código de Juízo, que podem variar dependendo do caso específico. Além disso, é fundamental considerar a possibilidade de acordo extrajudicial de responsabilidade penal, que pode levar a uma pena mais leniente.

É importante lembrar que a lesão corporal leve pode ser considerada como um crime grave, portanto, é importante buscar um advogado especializado em direito penal para acompanhar o processo e buscar as melhores opções de defesa.


## 6.3 Um astrofísico

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        # max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva como se fosse um astrofísico. Usando o idioma português, me explique por que o universo está expandindo.'

avaliarTexto(texto)

Resposta: 
Oi there! *adjusts glasses* As an astrophysicist, I'm here to tell you that the universe is expanding at an incredible rate! *excitedly* It's a bit mind-boggling, but bear with me here.

You see, the universe is made up of billions of galaxies, each containing billions of stars. And within those stars, there are nuclear reactions happening that create energy. That energy is what keeps the stars shining bright! *nods*

But here's the thing: the universe is expanding at an ever-accelerating rate. *leans in* It's like a big balloon that's being inflated, and the more it expands, the faster it gets! *excitedly*

Now, you might be wondering why this is happening. Well, the short answer is: we're not entirely sure! *winks* But there are a few theories. Some scientists think it could be due to the energy released by the Big Bang, which is the event that marked the beginning of the universe. *nods*

Others think it could be due to the presence of dark matter, a mysterious substance 